# CRI iAtlas notebooks

## Working with immune feature trends.

    Repo: https://github.com/CRI-iAtlas/iatlas-notebooks/ 
    Notebook: immune_feature_trends.ipynb 
    Date: August 16, 2021 
    Author: David L Gibbs 

# Immune Feature Trends Module

This is an [R Markdown](http://rmarkdown.rstudio.com) Notebook. When you execute code within the notebook, the results appear beneath the code. 
Try executing this chunk by clicking the *Run* button within the chunk or by placing your cursor inside it and pressing *Ctrl+Shift+Enter*. 
First we are going to install the iAtlas-modules package to gain access to plotting functions.

In [ ]:
packages = c("magrittr", "wrapr", "dplyr", "feather", "tidyr")

sapply(packages, function(x) {
  if (!require(x,character.only = TRUE))
    install.packages(x)
    library(x,character.only = TRUE)
})

# and our iatlas package from github
if (!require(iatlas.modules)) {
  devtools::install_github("CRI-iAtlas/iatlas.modules")
  library(iatlas.modules)
}


We have a collection of helper functions in the 'notebook_functions.R' file.

The main plotting function we're using here is called 'plotly_violin'.
You can get help with '?plotly_bar'.

Here we'll use source to bring them in.

In [ ]:
library(iatlas.modules)
source('functions/notebook_functions.R')

# Distributions

# Tissue compartment fractions barplot

In this case we'll assume you have some immune features computed
for each sample.

We'll need a data.frame with the following columns in order:

Group         ## (could be anything, but immune subytpe is a good choice.)
fraction_type ## c(leukocyte_fraction, Tumor_fraction, Stromal_fraction)
fraction      ## a numeric value.

Here's a example simulation.

In [ ]:

df <- data.frame(
  GROUP = sample(x=c('A1','A2','A3','A4','A5','A6'), size=100, replace=T),
  fraction_type = sample( c('leukocyte_fraction', 'Wound Healing'), size=100, replace=T),
  fraction=rnorm(n=100, mean = 0.5, sd = 0.2)
)

head(df)

The violin function takes a very specific format, so we'll transform the 
data.frame from above using the 'build_cellcontent_barplot_df2' function.
It essentially gives it a tidy format with generic column names.

# How to use the function

Parameters available:

  - y_col:  what do you want to plot?
  
  - scale_func_choice: the values can be transformed by log2, log2+1, log10, log10+1, or None (default)
  
  - reorder_func_choice: reorders the x-axis depending on the: mean, median, max, min, or None (default)
  
  - fill_colors
  
  - points
  
  - showlegend: T or F shows the legend (default is on)

  - plot labels: ylab, xlab, title



In [ ]:


  notebook_violinplot(
    data = df, 
    y_col = 'Wound Healing',        # plot this feature
    scale_func_choice = 'Log2 + 1', # how to scale the feature
    reorder_func_choice = "Mean",   # how to reorder the feature
    fill_colors = NULL,             # color scheme
    points = F,                     # show outliers
    showlegend = T,  # T or F       # show legend
    ylab = 'Wound Healing',             
    xlab = 'Immune Subtype',
    title = 'Leukocyte Fraction by Immune Subtype')


In [ ]:

  notebook_boxplot(
    data = df, 
    y_col = 'leukocyte_fraction',  # filter to this feature
    scale_func_choice = 'None', # how to scale the feature
    reorder_func_choice = "Mean",   # how to reorder the feature
    fill_colors = NULL,
    ylab = 'Leukocyte Fraction', 
    xlab = 'Immune Subtype',
    title = 'Leukocyte Fraction by Immune Subtype')


# Plot your results with iAtlas results.


Let us then create a new data.frame that can be merged with the simulated data above.
Since we're grouping the barplots, we want to make sure the iatlas groups have
different names compared to your groups... even if they represent fundementaly
the same thing (i.e. C2).

In [ ]:

iatlas_df <- get_iatlas_feature(group='Subtype_Immune_Model_Based', 
                                group_filter=c('C1', 'C2'),
                                group_mod = 'iAtlas_',
                                feature_name = 'leukocyte_fraction',
                                fraction_format = T)


# and we can merge in our data from above
new_df <- rbind(df, iatlas_df)

new_df

In [ ]:

  notebook_boxplot(
    new_df, 
    y_col = 'leukocyte_fraction',  # filter to this feature
    scale_func_choice = 'None', # how to scale the feature
    reorder_func_choice = "Mean",   # how to reorder the feature
    fill_colors = NULL,
    ylab = 'Leukocyte Fraction', 
    xlab = 'Groups',
    title = 'Leukocyte Fraction by Immune Subtype')


Or by study:

In [ ]:

iatlas_df <- get_iatlas_feature(group='Study', 
                                group_filter=c('KICH', 'KIRC'),
                                group_mod = 'iAtlas_',
                                feature_name = 'leukocyte_fraction',
                                fraction_format = T)

# and we can merge in our data from above
new_df <- rbind(df, iatlas_df)

notebook_boxplot(
    new_df, 
    y_col = 'leukocyte_fraction',  # filter to this feature
    scale_func_choice = 'None', # how to scale the feature
    reorder_func_choice = "Mean",   # how to reorder the feature
    fill_colors = NULL,
    ylab = 'Leukocyte Fraction', 
    xlab = 'Groups',
    title = 'Leukocyte Fraction by Immune Subtype')


# Correlation Heatmaps

In this case we'll assume you have some immune features computed for each sample
with TCGA sample IDs.

What we want:
have some new data that's calculated from TCGA

Want to correlate that with a set of vars.
and plot

In [ ]:
# get a list of feature sets
list_iatlas_feature_sets()


In [ ]:

# This will be the Response variable that will be correlated with a set of iatlas features
response_df <- get_iatlas_feature(group='Study', 
                                group_filter=c('KICH', 'KIRC'),
                                group_mod = '',
                                feature_name = 'leukocyte_fraction',
                                return_barcodes = T)

# then we need a table of Variables
var_df <- get_iatlas_feature_set(group='Study', 
                                group_filter=c('KICH', 'KIRC'),
                                group_mod = '',
                                feature_set = 'T Helper Cell Score', # feature set name # list_iatlas_feature_sets()
                                return_barcodes = T,
                                format = 'wide')

# next we need to compute the correlations between them
# Need to have reponse_df have GROUP first column
# Need a column called 'value'
# Need column called 'ParticipantBarcode' to match to TCGA
corr_mat <- notebook_corr(response_df, var_df)

# and then we can 
create_heatmap(corr_mat = corr_mat, title = 'Leukocyte fraction correlations')
